In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from main import get_normalized_probability, cal_most_prob_scene, reverse_relation
import json
import pulp
import numpy as np
from LPConstraints import get_clevr_block_world_constraint
from utils import process_coord_scene

In [143]:
with open('../results/clevr_block_val/constraint_1234/rel_scenes.json', 'r') as f:
    scenes = json.load(f)['scenes']

with open('../data/clevr_block/clevr_attr_map.json', 'r') as f:
    attr_map = json.load(f)

In [110]:
scenes = [process_coord_scene(scene, 'blockworld') for scene in scenes]

In [19]:
def get_attribute_variables(objects, attributes_map):
    variables = []
    probabilities = {}
    num_objects = len(objects)

    for name, values in attributes_map.items():
        for v, _ in enumerate(values):
            for i in range(num_objects):
                variable_name = f'{name}_{i}_{v}'
                variables.append(variable_name)
                probabilities[variable_name] = objects[i][name][v]
    return variables, probabilities

def get_relationship_variables(scene, relationships):
    variables = []
    probabilities = {}

    for rel in relationships:
        for target, sources in enumerate(scene['relationships'][rel]):
            for source, probability in sources:
                # TODO: change the prediction for below and above (currently flipped)
                variable_name = f'{rel}_{source}_{target}'
                variables.append(variable_name)
                probabilities[variable_name] = probability
    return variables, probabilities

def get_LP_problem(scene, attr_map, eps=1e-50):
    attr_variables, attr_probabilities = get_attribute_variables(scene['objects'], attr_map['attributes'])
    rel_variables, rel_probabilities = get_relationship_variables(scene, attr_map['relations'])


    attr_variables = pulp.LpVariable.dict('attr', attr_variables, 0, 1, pulp.LpBinary)
    rel_variables = pulp.LpVariable.dict('rel', rel_variables, 0, 1, pulp.LpBinary)

    attr_objective = [attr_variables[i] * np.log(max(attr_probabilities[i], eps)) for i in attr_variables.keys()]
    rel_objective = [rel_variables[i] * np.log(max(rel_probabilities[i], eps)) + (1 - rel_variables[i]) * np.log((max(1 - rel_probabilities[i], eps))) for i in rel_variables.keys()]

    prob = pulp.LpProblem("sceneGraphProblem", pulp.LpMaximize)
    prob += pulp.lpSum(attr_objective + rel_objective)
    return prob, attr_variables, rel_variables

def get_predicted_scene_for_block_world(variables, scene):
    predicted_scene = {
        'objects': [{'mask': o['mask']} for o in scene['objects']],
        'relationships': { rel: [[] for _ in range(len(scene['objects']))] for rel in attr_map['relations']}
    }

    for v in variables:
        if v.varValue:
            tokens = v.name.split('_')
            if tokens[0] == 'attr':
                predicted_scene['objects'][int(tokens[2])][tokens[1]] = attr_map['attributes'][tokens[1]][int(tokens[3])]
            elif tokens[0] == 'rel':
                predicted_scene['relationships'][tokens[1]][int(tokens[3])].append(int(tokens[2]))
    return predicted_scene

In [113]:
from tqdm import tqdm
M = 100
predicted_scenes = []
solver = pulp.GUROBI_CMD(path = 'C:/gurobi951/win64/bin/gurobi_cl.exe')

for scene in tqdm(scenes):
    prob, attr_variables, rel_variables = get_LP_problem(scene, attr_map)
    get_clevr_block_world_constraint(prob, scene, rel_variables, attr_variables, attr_map, include_small_obj=True, include_large_cube=True, include_bottom_stack=True, include_yellow=True)
    prob.solve(solver)
    predicted_scenes.append(get_predicted_scene_for_block_world(prob.variables(), scene))

100%|██████████| 6000/6000 [10:03<00:00,  9.94it/s]


In [114]:
with open('../results/clevr_block_val/constraint_1234/scene_prob_fix.json', 'w') as f:
    json.dump({'scenes': predicted_scenes}, f)



Experiment with Gurobi Constraint formulations

## Blocksworld

In [20]:
from LPProblem import BlocksworldLPProblem
from tqdm import tqdm

In [123]:
with open('../results/clevr_block_val/constraint_1234/constraint_1234/data_44/scenes_coord_prob.json', 'r') as f:
    scenes = json.load(f)['scenes']

with open('../data/clevr_block/clevr_attr_map.json', 'r') as f:
    attr_map = json.load(f)

In [124]:
scenes = [process_coord_scene(scene, 'blockworld') for scene in scenes]

In [125]:
problem_solver = BlocksworldLPProblem(attr_map, include_small_obj=True, include_large_cube=True, include_bottom_stack=True, include_yellow=True)

In [126]:
predicted_scenes = []
for scene in tqdm(scenes):
    predicted_scenes.append(problem_solver.solve_for_scene(scene))

100%|██████████| 2000/2000 [01:09<00:00, 28.74it/s]


In [127]:
with open('../results/clevr_block_val/constraint_1234/constraint_1234/data_44/rel_scenes_fix_coord.json', 'w') as f:
    json.dump({'scenes': predicted_scenes}, f)

## Clevr

In [79]:
from LPProblem import ClevrLPProblem
from tqdm import tqdm
import json
from utils import process_coord_scene_clevr

with open('../results/clevr_val/original/constraint_1234/data_125/s.json', 'r') as f:
    scenes = json.load(f)['scenes']

with open('G:/My Drive/Research/data/Clevr_mini/clevr_attr_map.json', 'r') as f:
    attr_map = json.load(f)

In [80]:
scenes = [process_coord_scene_clevr(scene) for scene in scenes]


KeyError: 'x'

In [81]:
problem_solver = ClevrLPProblem(attr_map,color_between=True, material_order=True, large_cube=True, object_behind=True)

In [82]:
predicted_scenes = []
for scene in tqdm(scenes):
    predicted_scenes.append(problem_solver.solve_for_scene(scene))

100%|██████████| 2000/2000 [01:09<00:00, 28.78it/s]


In [83]:
with open('../results/clevr_val/original/constraint_1234/data_125/rel_scenes_fix_c1234.json', 'w') as f:
    json.dump({'scenes': predicted_scenes}, f)

# VG

In [188]:
from LPProblem import VGLPProblem
from clevr_block_gen.constraints import get_vg_constraint_map
import torch
import json
from tqdm.notebook import tqdm


In [189]:
constraints = get_vg_constraint_map()

In [190]:
with open('../results/vg/probablistic_scenes.pytorch', 'rb') as f:
    scenes = torch.load(f)['scenes']

with open('../results/vg/schema.json', 'r') as f:
    schema = json.load(f)

In [195]:
problem_solver = VGLPProblem(attr_map=schema, opposite=True, person=True, transitivity=True)

In [197]:
predicted_scenes = []

for scene in tqdm(scenes[:100]):
    predicted_scenes.append(problem_solver.solve_for_scene(scene))

  0%|          | 0/100 [00:00<?, ?it/s]

In [193]:
violations = []
for i, scene in enumerate(predicted_scenes):
    for name, constraint in constraints.items():
        if not constraint.evaluate(scene):
            violations.append((i, name))
print(len(violations))

0


In [180]:
schema['idx_to_predicate']

{'1': 'above',
 '2': 'across',
 '3': 'against',
 '4': 'along',
 '5': 'and',
 '6': 'at',
 '7': 'attached to',
 '8': 'behind',
 '9': 'belonging to',
 '10': 'between',
 '11': 'carrying',
 '12': 'covered in',
 '13': 'covering',
 '14': 'eating',
 '15': 'flying in',
 '16': 'for',
 '17': 'from',
 '18': 'growing on',
 '19': 'hanging from',
 '20': 'has',
 '21': 'holding',
 '22': 'in',
 '23': 'in front of',
 '24': 'laying on',
 '25': 'looking at',
 '26': 'lying on',
 '27': 'made of',
 '28': 'mounted on',
 '29': 'near',
 '30': 'of',
 '31': 'on',
 '32': 'on back of',
 '33': 'over',
 '34': 'painted on',
 '35': 'parked on',
 '36': 'part of',
 '37': 'playing',
 '38': 'riding',
 '39': 'says',
 '40': 'sitting on',
 '41': 'standing on',
 '42': 'to',
 '43': 'under',
 '44': 'using',
 '45': 'walking in',
 '46': 'walking on',
 '47': 'watching',
 '48': 'wearing',
 '49': 'wears',
 '50': 'with'}

In [194]:
with open('../results/vg/scene_fixed.json', 'w') as f:
    json.dump({'scenes': predicted_scenes}, f)

In [77]:
from measurements import ProblemDomain

330

In [147]:
from itertools import permutations
list(permutations(range(10), 3))

[(0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 1, 5),
 (0, 1, 6),
 (0, 1, 7),
 (0, 1, 8),
 (0, 1, 9),
 (0, 2, 1),
 (0, 2, 3),
 (0, 2, 4),
 (0, 2, 5),
 (0, 2, 6),
 (0, 2, 7),
 (0, 2, 8),
 (0, 2, 9),
 (0, 3, 1),
 (0, 3, 2),
 (0, 3, 4),
 (0, 3, 5),
 (0, 3, 6),
 (0, 3, 7),
 (0, 3, 8),
 (0, 3, 9),
 (0, 4, 1),
 (0, 4, 2),
 (0, 4, 3),
 (0, 4, 5),
 (0, 4, 6),
 (0, 4, 7),
 (0, 4, 8),
 (0, 4, 9),
 (0, 5, 1),
 (0, 5, 2),
 (0, 5, 3),
 (0, 5, 4),
 (0, 5, 6),
 (0, 5, 7),
 (0, 5, 8),
 (0, 5, 9),
 (0, 6, 1),
 (0, 6, 2),
 (0, 6, 3),
 (0, 6, 4),
 (0, 6, 5),
 (0, 6, 7),
 (0, 6, 8),
 (0, 6, 9),
 (0, 7, 1),
 (0, 7, 2),
 (0, 7, 3),
 (0, 7, 4),
 (0, 7, 5),
 (0, 7, 6),
 (0, 7, 8),
 (0, 7, 9),
 (0, 8, 1),
 (0, 8, 2),
 (0, 8, 3),
 (0, 8, 4),
 (0, 8, 5),
 (0, 8, 6),
 (0, 8, 7),
 (0, 8, 9),
 (0, 9, 1),
 (0, 9, 2),
 (0, 9, 3),
 (0, 9, 4),
 (0, 9, 5),
 (0, 9, 6),
 (0, 9, 7),
 (0, 9, 8),
 (1, 0, 2),
 (1, 0, 3),
 (1, 0, 4),
 (1, 0, 5),
 (1, 0, 6),
 (1, 0, 7),
 (1, 0, 8),
 (1, 0, 9),
 (1, 2, 0),
 (1, 2, 3),
 (1, 2, 4),
 (1,